<a href="https://colab.research.google.com/github/teamgaon/KLUE/blob/main/220223_hj_eng_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A full training

Install the Transformers and Datasets libraries to run this notebook.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install transformers

     |████████████████████████████████| 311 kB 5.3 MB/s 
     |████████████████████████████████| 3.5 MB 58.1 MB/s 
     |████████████████████████████████| 134 kB 82.4 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 243 kB 81.7 MB/s 
     |████████████████████████████████| 1.1 MB 65.9 MB/s 
     |████████████████████████████████| 271 kB 80.7 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 144 kB 77.1 MB/s 
     |████████████████████████████████| 6.8 MB 69.6 MB/s 
     |████████████████████████████████| 895 kB 72.4 MB/s 
     |████████████████████████████████| 596 kB 81.3 MB/s 
     |████████████████████████████████| 1.2 MB 68.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 58 kB 3.2 MB/s 
     |████████████████████████████████| 149.9 

In [3]:
from accelerate import Accelerator

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from transformers import TrainingArguments, Trainer
from transformers import AutoModel,AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
import gc
from transformers import AdamW
from transformers import get_scheduler, get_cosine_with_hard_restarts_schedule_with_warmup
from tqdm.auto import tqdm
from datasets import load_metric
import warnings
warnings.filterwarnings('ignore')
from accelerate import notebook_launcher

In [5]:
train = pd.read_json('/content/drive/MyDrive/KLUE/multinli_1.0_train.jsonl',lines=True)
train = train[['sentence1', 'sentence2', 'gold_label']]
train.rename(columns = {'sentence1':'premise','sentence2':'hypothesis','gold_label':'label'},inplace=True)
train = train.reset_index(drop=True)
train['index'] = train.index
train = train.dropna()
train = train[:100000]

In [7]:
from transformers import AutoTokenizer

checkpoint = 'roberta-large'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [8]:
# train_dataset, eval_dataset = train_test_split(train, test_size=0.2, shuffle=False)

# tokenized_train = tokenizer(
#     list(train_dataset['premise']),
#     list(train_dataset['hypothesis']),
#     return_tensors="pt",
#     max_length=128, # Max_Length = 190
#     padding=True,
#     truncation=True,
#     add_special_tokens=True
# )

# tokenized_eval = tokenizer(
#     list(eval_dataset['premise']),
#     list(eval_dataset['hypothesis']),
#     return_tensors="pt",
#     max_length=128,
#     padding=True,
#     truncation=True,
#     add_special_tokens=True
# )

In [9]:
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, pair_dataset, label):
        self.pair_dataset = pair_dataset
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['labels'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):
        return len(self.label)

In [10]:
def label_to_num(label):
    label_dict = {"entailment": 0, "contradiction": 1, "neutral": 2, "answer": 3}
    num_label = []

    for v in label:
        num_label.append(label_dict[v])
    
    return num_label

# train_label = label_to_num(train['label'].values)
# eval_label = label_to_num(eval_dataset['label'].values)

In [11]:
# train_dataset = BERTDataset(tokenized_train, train_label)
# eval_dataset = BERTDataset(tokenized_eval, eval_label)

In [12]:
# train_dataset[0]

In [13]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [14]:
# def training_function():
#   accelerator = Accelerator()

#   train_dataloader = torch.utils.data.DataLoader(
#                     train_dataset, 
#                     batch_size=16, sampler=train_subsampler, num_workers=2)
#   eval_dataloader = torch.utils.data.DataLoader(
#                     train_dataset,
#                     batch_size=16, sampler=test_subsampler, num_workers=2)

#   config = AutoConfig.from_pretrained(checkpoint)
#   config.num_labels = 3
#   model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

#   optimizer = AdamW(model.parameters(), lr=	1e-5)

#   model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

#   num_epochs = 5
#   num_training_steps = num_epochs * len(train_dataloader)
#   progress_bar = tqdm(range(num_training_steps))
#   lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
#       optimizer=optimizer,
#       num_warmup_steps=1,
#       num_training_steps=num_training_steps,
#   )

#   for epoch in range(num_epochs):
#       train_acc = 0.0
#       test_acc = 0.0

#       model.train()
#       for batch_id, batch in enumerate(train_dataloader):
#           outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
#           loss = F.cross_entropy(outputs[0], batch['labels'])
#           accelerator.backward(loss)

#           optimizer.step()
#           lr_scheduler.step()
#           optimizer.zero_grad()
#           progress_bar.update(1)
#           train_acc += calc_accuracy(outputs.logits, batch['labels'])
#       print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

#       model.eval()
#       for batch_id, batch in enumerate(eval_dataloader):
#         with torch.no_grad():
#             outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

#         test_acc += calc_accuracy(outputs.logits, batch['labels'])
#       print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
#       gc.collect()
#   accelerator.wait_for_everyone()
#   unwrapped_model = accelerator.unwrap_model(model)
#   accelerator.save(unwrapped_model.state_dict(), '/content/drive/MyDrive/KLUE/model.pt')

In [15]:
# # kfold = StratifiedKFold(n_splits=5, shuffle=True)

# # Start print
# # print('--------------------------------')

# # K-fold Cross Validation model evaluation
# def training_function():
#   kfold = StratifiedKFold(n_splits=5, shuffle=True)

#   print('--------------------------------')

#   for fold, (train_ids, test_ids) in enumerate(kfold.split(train, train['label'])):
#     # print('')
#     print(f'FOLD {fold}')

#     train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
#     test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

#     train_label = label_to_num(train['label'].values)

#     tokenized_train = tokenizer(
#       list(train['premise']),
#       list(train['hypothesis']),
#       return_tensors="pt",
#       max_length=128, # Max_Length = 190
#       padding=True,
#       truncation=True,
#       add_special_tokens=True
#       )
    
#     train_dataset = BERTDataset(tokenized_train, train_label)

#     # def training_function():
#     accelerator = Accelerator()

#     train_dataloader = torch.utils.data.DataLoader(
#                       train_dataset, 
#                       batch_size=8, sampler=train_subsampler)
#     eval_dataloader = torch.utils.data.DataLoader(
#                       train_dataset,
#                       batch_size=8, sampler=test_subsampler)

#     config = AutoConfig.from_pretrained(checkpoint)
#     config.num_labels = 3
#     model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

#     optimizer = AdamW(model.parameters(), lr=	1e-5)

#     model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

#     num_epochs = 5
#     num_training_steps = num_epochs * len(train_dataloader)
#     progress_bar = tqdm(range(num_training_steps))
#     lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
#         optimizer=optimizer,
#         num_warmup_steps=1,
#         num_training_steps=num_training_steps,
#     )

#     for epoch in range(num_epochs):
#         train_acc = 0.0
#         test_acc = 0.0

#         model.train()
#         for batch_id, batch in enumerate(train_dataloader):
#             outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
#             loss = F.cross_entropy(outputs[0], batch['labels'])
#             accelerator.backward(loss)

#             optimizer.step()
#             lr_scheduler.step()
#             optimizer.zero_grad()
#             progress_bar.update(1)
#             train_acc += calc_accuracy(outputs.logits, batch['labels'])
#         print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

#         model.eval()
#         for batch_id, batch in enumerate(eval_dataloader):
#           with torch.no_grad():
#               outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

#           test_acc += calc_accuracy(outputs.logits, batch['labels'])
#         print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
#         gc.collect()
#     accelerator.wait_for_everyone()
#     unwrapped_model = accelerator.unwrap_model(model)
#     accelerator.save(unwrapped_model.state_dict(), '/content/drive/MyDrive/KLUE/model' + str(fold) + '.pt')

In [20]:
def training_function():
  accelerator = Accelerator()

  train_dataloader = torch.utils.data.DataLoader(
                    train_dataset, 
                    batch_size=8, sampler=train_subsampler)
  eval_dataloader = torch.utils.data.DataLoader(
                    train_dataset,
                    batch_size=8, sampler=test_subsampler)

  config = AutoConfig.from_pretrained(checkpoint)
  config.num_labels = 3
  model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

  optimizer = AdamW(model.parameters(), lr=	1e-5)

  model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

  num_epochs = 5
  num_training_steps = num_epochs * len(train_dataloader)
  progress_bar = tqdm(range(num_training_steps))
  lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
      optimizer=optimizer,
      num_warmup_steps=1,
      num_training_steps=num_training_steps,
  )

  for epoch in range(num_epochs):
      train_acc = 0.0
      test_acc = 0.0

      model.train()
      for batch_id, batch in enumerate(train_dataloader):
          outputs = model(batch['input_ids'], batch['attention_mask'])
          loss = F.cross_entropy(outputs[0], batch['labels'])
          accelerator.backward(loss)

          optimizer.step()
          lr_scheduler.step()
          optimizer.zero_grad()
          progress_bar.update(1)
          train_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

      model.eval()
      for batch_id, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            outputs = model(batch['input_ids'], batch['attention_mask'])

        test_acc += calc_accuracy(outputs.logits, batch['labels'])
      print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
      gc.collect()
  accelerator.wait_for_everyone()
  unwrapped_model = accelerator.unwrap_model(model)
  accelerator.save(unwrapped_model.state_dict(), '/content/drive/MyDrive/KLUE_model/hj_model' + str(fold) + '.pt')

In [ ]:
# kfold = StratifiedKFold(n_splits=5, shuffle=True)

# Start print
# print('--------------------------------')

# K-fold Cross Validation model evaluation

kfold = StratifiedKFold(n_splits=5, shuffle=True)

print('--------------------------------')

tokenized_train = tokenizer(
  list(train['premise']),
  list(train['hypothesis']),
  return_tensors="pt",
  max_length=128, # Max_Length = 190
  padding=True,
  truncation=True,
  add_special_tokens=True
  )
  
for fold, (train_ids, test_ids) in enumerate(kfold.split(train, train['label'])):
  # print('')
  print(f'FOLD {fold}')

  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

  train_label = label_to_num(train['label'].values)

  train_dataset = BERTDataset(tokenized_train, train_label)

  notebook_launcher(training_function)

--------------------------------
FOLD 0
Launching a training on 8 TPU cores.


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classi

  0%|          | 0/6250 [00:00<?, ?it/s]

  0%|          | 0/6250 [00:00<?, ?it/s]

  0%|          | 0/6250 [00:00<?, ?it/s]

...

  0%|          | 0/6250 [00:00<?, ?it/s]

  0%|          | 0/6250 [00:00<?, ?it/s]

  0%|          | 0/6250 [00:00<?, ?it/s]

  0%|          | 0/6250 [00:00<?, ?it/s]

epoch 1 train acc 0.8117
epoch 1 train acc 0.8078
epoch 1 train acc 0.8145
epoch 1 train acc 0.8137
epoch 1 train acc 0.8128
epoch 1 train acc 0.8099
epoch 1 train acc 0.8044
epoch 1 train acc 0.8112
epoch 1 test acc 0.8905750798722045
epoch 1 test acc 0.889776357827476
epoch 1 test acc 0.8582268370607029
epoch 1 test acc 0.8853833865814696
epoch 1 test acc 0.8690095846645367
epoch 1 test acc 0.8821884984025559
epoch 1 test acc 0.8837859424920128
epoch 1 test acc 0.8686102236421726
epoch 2 train acc 0.9026
epoch 2 train acc 0.9034
epoch 2 train acc 0.9019
epoch 2 train acc 0.8988
epoch 2 train acc 0.9002
epoch 2 train acc 0.8998
epoch 2 train acc 0.9038
epoch 2 train acc 0.9017
epoch 2 test acc 0.8857827476038339
epoch 2 test acc 0.8857827476038339
epoch 2 test acc 0.8801916932907349
epoch 2 test acc 0.8881789137380192
epoch 2 test acc 0.8885782747603834
epoch 2 test acc 0.8793929712460063
epoch 2 test acc 0.8789936102236422
epoch 2 test acc 0.8957667731629393


In [19]:
train_dataset[0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([    0,  9157, 16771, 13851,  6353,  2972, 16364,    34,    80,  3280,
         22735,   111,  1152,     8, 18947,     4,     2,     2, 41257,     8,
         18947,    32,    99,   146,  6353,  2972, 16364,   173,     4,  1437,
             2,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     

In [ ]:
print(my_variable)

In [ ]:
config = AutoConfig.from_pretrained(checkpoint)
config.num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

In [ ]:
accelerator = Accelerator()
model = accelerator.unwrap_model(model)
model.load_state_dict(torch.load('/content/drive/MyDrive/model/model.pt'))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
test_label = label_to_num(test['label'].values)

tokenized_test = tokenizer(
    list(test['premise']),
    list(test['hypothesis']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = BERTDataset(tokenized_test, test_label)

In [ ]:
dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

output_pred = []
output_prob = []

model, dataloader= accelerator.prepare(model, dataloader)

model.eval()

for i, data in enumerate(tqdm(dataloader)):
    with torch.no_grad():
        outputs = model(
            input_ids=data['input_ids'],
            attention_mask=data['attention_mask'],
            token_type_ids=data['token_type_ids']
        )
    logits = outputs[0]
    prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits, axis=-1)

    output_pred.append(result)
    output_prob.append(prob)
  
pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()
print(pred_answer)

In [ ]:
def num_to_label(label):
    label_dict = {0: "entailment", 1: "contradiction", 2: "neutral"}
    str_label = []

    for i, v in enumerate(label):
        str_label.append([i,label_dict[v]])
    
    return str_label

answer = num_to_label(pred_answer)
print(answer)

In [ ]:
df = pd.DataFrame(answer, columns=['index', 'label'])

df.to_csv('/content/drive/MyDrive/KLUE/submission.csv', index=False)

print(df)